# Homework 1

## 1.1

Introspection would not be adequate in and of itself to help with modelling human cognitive processes because it is seen
from an entirely human perspective. It would take for granted all of the mechanisms which we as humans don't even have
to consciously think about. For example, we as humans solve simple problems like eating when we're hungry and going to
sleep when we're tired every day. We never have to explicitly think about how to eat we have to go to the pantry, decide
what we "want" to eat, then prepare it, put it on a plate, and put it in our mouths. We also don't have to think about
how to digest that food and get the essential nutrients that our body needs to survive. Through introspection, it is
nearly impossible to come up with all the steps it took to solve a problem.

Introspection can, however, be used to get the general idea of how a cognitive process works and then adapt it to work
with a machine. Take solving sudoku with constraint satisfaction, for example. Introspection was most likely used to
determine that humans often solve sudoku using constraint satisfaction, and then that was able to be applied to a
machine in a similar fashion. So, introspection can be helpful in modeling human cognitive processes to an extent, but
it often misses some key mechanisms that humans don't have to think about.

## 1.2

In my TSP problem formulation, the state is represented by an ordered list of numbers representing cities in which each
city number appears once. The actions are performed by swapping two city numbers in the list.

In [2]:
from search import Problem


class TSP(Problem):
    """An implementation of the Travelling Salesman Problem for local search.

        State representation:
            [c1, c2, ..., cn] gives the order of cities to be visited.
        Move representation:
            [c1, c2]: Swap the two cities
        """

    def __init__(self, graph, initial):
        self.graph = graph
        self.initial = initial

    def actions(self, state):
        """ Actions are all of the permutations of swapping two cities"""
        actions = []
        for i in range(len(state)):
            for j in range(len(state)):
                if i != j:
                    actions.append([i, j])
        return actions

    def result(self, state, move):
        """Makes the given move on a copy of the given state."""
        new_state = state[:]
        new_state[move[0]] = state[move[1]]
        new_state[move[1]] = state[move[0]]
        return new_state

    def value(self, state):
        """The value is the sum of all the distances in the state multiplied by -1
            to allow for maximization
        """
        value = 0
        for i in range(len(state) - 1):
            value += self.graph[state[i]][state[i+1]]
        value += self.graph[state[-1]][state[0]]
        return -value


This sample city-domain creates a random number of cities (max 50) and assigns random distances between the cities.
The TSP is then able to be solved using both hill-climbing and simulated annealing.

In [3]:
from search import hill_climbing, simulated_annealing, exp_schedule
import random

numCities = random.randint(4, 50)

# Initialize distances to all 0
distances = []
for i in range(numCities):
    distances.append([])
    for j in range(numCities):
        distances[i].append(0)

# Populate distances randomly and symmetrically
for i in range(numCities):
    for j in range(numCities):
        if i > j:
            rand_dist = random.randint(1, 1000)
            distances[i][j] = distances[j][i] = rand_dist

initial_state = []
for i in range(numCities):
    initial_state.append(i)

p = TSP(distances, initial_state)

# Solve the problem using hill climbing.
hill_solution = hill_climbing(p)
print('\nHill-climbing:')
print('\tSolution:\t' + str(hill_solution))
print('\tValue:\t\t' + str(p.value(hill_solution)))

# Solve the problem using simulated annealing.
annealing_solution = \
    simulated_annealing(p, exp_schedule(k=20, lam=0.005, limit=10000))
print('\nSimulated annealing:')
print('\tSolution:\t' + str(annealing_solution))
print('\tValue:\t\t' + str(p.value(annealing_solution)))



Hill-climbing:
	Solution:	[0, 18, 2, 15, 12, 5, 13, 9, 8, 7, 10, 11, 6, 4, 14, 1, 16, 17, 3]
	Value:		-3124

Simulated annealing:
	Solution:	[0, 3, 17, 6, 4, 14, 12, 15, 8, 7, 10, 11, 2, 9, 13, 5, 1, 16, 18]
	Value:		-2978


Simulated annealing is generally able to perform better than hill-climbing on the TSP, but not always. There are so many
different combinations of paths to take once the number of cities is at least at 10, and a lot of them result in a
longer path, which causes some inconsistency in simulated annealing performing better than hill-climbing consistently.
However, simulated annealing is able to perform better than hill-climbing as a whole because there are lots of local
maxima in the search space causing hill-climbing to get stuck while simulated annealing is still able to potentially
find a better solution.

## 1.3

In my class scheduler problem formulation, the variables are the classes and the domains are the possible
faculty-time-classroom combinations. The Scheduler class takes the list of classes, faculty, times, and classrooms along
with a dictionary of class-professor assignments to formulate the CSP.

In [4]:
from csp import CSP


def scheduler_constraint(A, a, B, b):
    a_list = a.split('-')
    b_list = b.split('-')
    # Fail if the class is repeated
    if A == B:
        return False
    # Fail if the faculty member is scheduled for two classes at the same time
    if a_list[0] == b_list[0] and a_list[1] == b_list[1]:
        return False
    # Fail if the classroom is scheduled for two classes at the same time
    if a_list[1] == b_list[1] and a_list[2] == b_list[2]:
        return False
    return True


class Scheduler(CSP):
    """Build a class scheduler problem. The classes are the variables, the domains are the possible combinations
        of faculty, time, and classroom and the neighbors for each class is every other class"""

    def __init__(self, classes, faculty, assignments, times, classrooms):
        domains = {}
        neighbors = {}
        for cs_class in classes:
            domains[cs_class] = []
            neighbors[cs_class] = []
            for faculty_member in faculty:
                # Assign the faculty member
                if faculty_member == assignments[cs_class]:
                    for time in times:
                        for classroom in classrooms:
                            domains[cs_class].append(faculty_member + '-' + time + '-' + classroom)
        for cs_class in classes:
            for entry in neighbors:
                if cs_class != entry:
                    neighbors[entry].append(cs_class)

        CSP.__init__(self, classes, domains, neighbors, scheduler_constraint)
        

This sample scheduler takes in example classes, faculty, assignments, times, and classrooms and uses backtracking
search to successfully solve the problem.

In [5]:
from csp import backtracking_search


def print_result(result):
    for k in result:
        v_list = result[k].split('-')
        print('Class:', k)
        print('\tFaculty:', v_list[0])
        print('\tTime:', v_list[1])
        print('\tClassroom:', v_list[2])


cs_classes = ['cs108', 'cs112', 'cs212', 'cs214', 'cs262', 'cs232', 'cs384']
cs_faculty = ['vtn2', 'adams', 'hplantin', 'kvlinden', 'dschuurman']
cs_assignments = {
    'cs108': 'vtn2',
    'cs112': 'adams',
    'cs212': 'hplantin',
    'cs214': 'adams',
    'cs262': 'kvlinden',
    'cs232': 'vtn2',
    'cs384': 'dschuurman'
}
cs_times = ['mwf800', 'mwf900', 'mwf1030', 'mwf1130']
cs_classrooms = ['nh253', 'sb382']
scheduler = Scheduler(cs_classes, cs_faculty, cs_assignments, cs_times, cs_classrooms)
result = backtracking_search(scheduler)
print_result(result)

Class: cs108
	Faculty: vtn2
	Time: mwf800
	Classroom: nh253
Class: cs112
	Faculty: adams
	Time: mwf800
	Classroom: sb382
Class: cs212
	Faculty: hplantin
	Time: mwf900
	Classroom: nh253
Class: cs214
	Faculty: adams
	Time: mwf900
	Classroom: sb382
Class: cs262
	Faculty: kvlinden
	Time: mwf1030
	Classroom: nh253
Class: cs232
	Faculty: vtn2
	Time: mwf1030
	Classroom: sb382
Class: cs384
	Faculty: dschuurman
	Time: mwf1130
	Classroom: nh253


I formulated this problem using the classes as the variables and the faculty-time-classroom combinations as the domains
because it was the easiest way to handle all of the lists of data used to formulate the problem. The classes list was
able to directly become the variables and the other lists were able to be easily looped through to determine the
possible combinations for each class.